In [1]:
import os

# for rendering
from IPython.core.display import HTML, Markdown

# for data analysis
import numpy as np
import datetime

# for plotting
import plotly.graph_objects as go
import svgutils.compose as sc

# figure/table number counter
from smartydoc.decorator import FigCounter

# for long table
import smartydoc.matplot as sdplot

In [2]:
# var for iteration
# this ID must be an unique index of a student / school,
# you can fetch ALL needed data used in this notebook with this ID. 
#iter_id = os.getenv('ITERID')
iter_id = 'sample12345'

# helper var for development
# if False, only display image embeded in the notebook,
# if True, the code would save the image as a SVG file for final report.
PRODUCTION_PHASE = True

# figure/table counter
fig_counter = FigCounter('图', font_size=15, font_family='SimHei')
tbl_counter = FigCounter('表', font_size=15, font_family='SimHei')


In [3]:
# image dir config
material_dir = os.path.join(os.path.curdir, 'imgs', 'common')
img_dir = os.path.join(os.path.curdir, 'imgs', iter_id)
if not os.path.exists(img_dir):
    os.makedirs(img_dir, mode=0o755)

# helpers for image display
def display_image(img_file, cls='medium'):
    """
    img_file: a image file path
    cls: small, medium, large, sign, or qrcode
    """
    # os.path.relpath(path) -> 返回path的相对路径，保证文件整体移动后仍可以找到图片
    if cls:
        html_str = '<div class="%s"><img src="%s" alt="image"></div>\n'%(cls, os.path.relpath(img_file))
    else: 
        html_str = '<div class="medium"><img src="%s" alt="image"></div>\n'%(os.path.relpath(img_file))
    display(HTML(html_str))

In [4]:
# helper for cover generation
# XXX: add more info
def gen_cover(cover_cfg):
    html_content = []
    if 'main_cn' in cover_cfg:
        html_content.append('<h1 id="cover">%s</h1>'%(cover_cfg['main_cn']))
    if 'school' in cover_cfg:
        html_content.append('<school>%s</school>'%(cover_cfg['school']))
    if 'stu_name' in cover_cfg:
        html_content.append('<stuname>%s</stuname>'%(cover_cfg['stu_name']))
    if 'stu_class' in cover_cfg:
        html_content.append('<stuclass>%s</stuclass>'%(cover_cfg['stu_class']))
    if 'company' in cover_cfg:
        html_content.append('<address>%s</address>'%(cover_cfg['company']))
    if 'test_date' in cover_cfg:
        html_content.append('<testdate>%s</testdate>'%(cover_cfg['test_date']))

    display(HTML('\n'.join(html_content)))

In [5]:
# cover config
today = datetime.datetime.today()
cover_cfg = {'main_cn': '使用Jupyter Notebook<br>撰写数据分析报告',
             'company': 'SmartyDoc项目组',
             'test_date': str(today.year)+'年'+str(today.month)+'月',
            }
gen_cover(cover_cfg)

## 前言

> Jupyter Notebook是基于网页的用于交互计算的应用程序，其可被应用于全过程计算：开发、文档编写、运行代码和展示结果。
>
> <rightalign> — Jupyter Notebook 官方介绍 </rightalign>

In [6]:
display_image(os.path.join(material_dir, 'jupyter_home.png'))

简而言之，Jupyter Notebook是以网页的形式存储，可以在网页页面中直接编写和运行代码，代码的运行结果也会直接在代码块下显示。如在编程过程中需要编写说明文档，可在同一个页面中直接编写，便于作及时的说明和解释。

基于Jupyter Notebook具备富文档和标准化的特性，我们也希望可以利用它生成美观的数据分析报告。**SmartyDoc**就是为此而生。利用**SmartyDoc**提供的文本标准化流程，我们可以将包含图文的Jupyter Notebook文件转存为具备特定层级结构的html文件，配合适当的css文件以及**weasyprint**工具，即可产生PDF格式的报告文档。

要顺利使用**SmartyDoc**完成报告，要求使用者掌握Python和MarkDown两种语言。

下面将具体介绍如何利用这套工具撰写报告文档。

## 规划报告的逻辑结构

在撰写数据分析报告时，把报告的逻辑结构规划好，将报告内容划分为几个相对独立、且逻辑连贯的章节，会让后续工作更加顺利。如整篇报告可以包含几大**章**，每**章**可以包含多个**小节**，每个**小节**可以进一步划分为多个**子节**。

在**SmartyDoc**的框架内，文档的层级结构基于MarkDown语言中的*Heading*实现，即报告的题目为一级标题，用 `# 报告题目` 这种形式实现，各个章节的标题为二级标题，用 `## 章节标题` 的形式实现，以下可以继续出现三级、四级...等不同等级的章节。

在撰写报告时，报告题目的样式以一段程序自动生成，用户只需提供报告封面的内容即可，具体可以参考此ipynb文件的*Cell 4*和*Cell 5*中的内容。需要用户自己写的报告章节结构，以二级标题为最高层级。

在每个章节下，可以出现文字、表格和图片等信息。为了保证用MarkDown语言撰写的报告内容可以正常显示，需要将对应*Cell*的语言设置为`MarkDown`。

### 显示Python程序中输出的文本

在进行数据分析时，我们经常需要根据分析结果打印对应的数据和结果描述，因此需要保证Python程序中打印的文字也能够正常显示在报告正文中。在**SmartyDoc**的框架下，我们可以使用如下方式显示Python程序中输出的文本。

In [7]:
some_score = 78
display(Markdown('打印Python程序中的输出文本 ... 变量的数值是 %s。'%(some_score)))

打印Python程序中的输出文本 ... 变量的数值是 78。

### 使用toc2插件管理文档目录

Jupyter Notebook是一个很棒的教学、探索和编程环境，但其功能仍存在很多不足。幸好，它允许我们使用一些插件来扩展它的功能。其中有一个插件叫Table of Contents (2) (toc2)，可以为Jupyter Notebook提供目录。可以在Jupyter的Nbextensions扩展管理页内，通过勾选框启用扩展。

点击按钮栏最后部的图标，目录列表会以左侧边栏的形式显示，toc2的使用效果见下图。


In [8]:
display_image(os.path.join(material_dir, 'jupyter_toc2.png'))

为了与**SmartyDoc**的框架兼容，需要对toc2扩展进行一些设置，可以通过点击目录列表上部的`齿轮图标`进行设置(如下图)，并将`h1`级标题排除出目录范畴（在**SmartyDoc**的框架下，章节结构以二级标题为最高层级）。

In [9]:
display_image(os.path.join(material_dir, 'jupyter_toc2_config.png'))

## 在报告中插入图表

图表可以让报告内容清晰明了。

在Jupyter Notebook中，用户可以直接将图片或表格嵌入到文本中。但为了便于将报告文本中所包含的文字、图片和表格都顺利地转换为HTML或PDF等格式，在**SmartyDoc**框架下，要求用户将图片和表格保存成图片文件，并在ipynb文本中进行引用，或直接在ipynb文件中以html格式保存。

为了方便报告内容的撰写，可以使用变量`PRODUCTION_PHASE`来控制图片在ipynb文件中保存的形式。具体可以参考*Cell 2*中的变量定义方式和下面的例子。

### 使用的工具包

为了得到美观且样式丰富的数据图，这里建议主要使用*plotly*工具包作图，并将图片保存为SVG格式，以保证在不同设备和缩放尺度下的图片质量。

对于一些非常具有设计感但难以通过*plotly*简单实现的图片，可以使用*svgutils*工具包中的图片组合功能，对图片进行重组和编辑。

具体使用样例请参考下面的示例。

### 自动添加图表编号

在撰写报告时会产生大量的图片，要准确标注每张图的编号将会消耗很大的人力，因此我们将这项工作交给程序自动完成。

在**SmartyDoc**中的`decorator`工具包中，提供了`FigCounter`类用来对图表编号进行自动计数。这个类的初始化方法请见 *Cell 2*，在初始化时需要提供图表名称的起始词，如图1，图2，...中的"图"字，以及设置文字的字号和字体等。初始化后，具体的使用方法请参考下文的示例。

### 图表示例

#### 在报告中显示图片

In [10]:
compose_fig = fig_counter.add_title(os.path.join(material_dir, 'jupyter_home.png'), '插入的png图片')
# you can append this part into your code
if PRODUCTION_PHASE:
    compose_fig.save(os.path.join(img_dir, 'inserted_png.svg'))
    display_image(os.path.join(img_dir, 'inserted_png.svg'))
else:
    display(compose_fig)

#### 利用Plotly画数字表-1

In [11]:
# table plot
fig = go.Figure(data=[go.Table(
    header=dict(values=['<b>学校名称</b>', '<b>总人数</b>',
                        '<b>男生</b>', '<b>女生</b>', '<b>班级数</b>'],
                line_color='darkslategray',
                fill_color='lightskyblue',
                font_family='SimHei',
                font=dict(color='white', size=14),
                height=30,
                align='center'),
    cells=dict(values=[['A学校', 'B学校', 'C学校'], # 1st column
                       ['100人', '90人', '80人'],  # 2nd column
                       ['60人', '30人', '70人'],   # 3rd column
                       ['40人', '60人', '10人'],   # 4th column
                       [10, 15, 6],               # 5th column
                      ],
               line_color='darkslategray',
               fill_color='lightcyan',
               font_family='SimHei',
               font_size=14,
               height=30,
               align='left'))
])

# 这里为表格添加表名和表的编号
fig.update_layout(width=600, height=320)
fig = tbl_counter.add_title(fig, '人数统计', y_pos=0)

# 为了方便撰写文档时，实时看到修改的结果，建议通过变量PRODUCTION_PHRASE设置撰写的状态，
# 将PRODUCTION_PHRASE设为False，直接将生成的图片嵌在Notebook中，设为True则保存为图片
# 由于浏览器自己的缓存功能，如果总是保存为图片文件，会出现无论如何修改，显示
# 出的图都不变的情况
# 注意：建议在每个画图函数后都添加这一段，通过文件开头的PRODUCTION_PHRASE变量设置全局的状态
if PRODUCTION_PHASE:
    img_file = os.path.join(img_dir, 'img1.svg')
    fig.write_image(img_file, scale=1)
    display_image(img_file)
else:
    fig.show()


#### 利用Plotly画数字表-2

In [12]:
fig = go.Figure(data=[go.Table(
    header=dict(values=['<b>平均<br>等级</b>',
                        '<b>文理<br>倾向</b>',
                        '<b>学科发展<br>均衡程度</b>',
                        '<b>A等级<br>学生人数</b>',
                        '<b>B等级<br>学生人数</b>',
                        '<b>全市<br>排名</b>',
                        '<b>全省<br>排名</b>',
                       ],
                font_family='SimHei',
                font=dict(color='black', size=14),
                height=30,
                align='center'),
    cells=dict(values=[['B'],
                       ['综合'],
                       ['较为均衡'],
                       ['54人'],
                       ['0人'],
                       ['9/61'],
                       ['22/287'],
                      ],
               font_family='SimSun',
               font_size=14,
               height=30,
               align='center'))
])
fig.update_layout(width=800, height=400)
fig = tbl_counter.add_title(fig, '结果统计', y_pos=0)

if PRODUCTION_PHASE:
    img_file = os.path.join(img_dir, 'img2.svg')
    fig.write_image(img_file, scale=1)
    display_image(img_file)
else:
    fig.show()


#### 利用Plotly画雷达图

In [13]:
labels = ['Dim1', 'Dim2', 'Dim3', 'Dim4', 'Dim5']
r = np.random.rand(5).tolist()
fig = go.Figure()
fig.add_trace(go.Scatterpolar(r=r, theta=labels, fill='toself', name='Test'))
fig.update_layout(polar=dict(radialaxis=dict(visible=True, range=[0, 1])), showlegend=False)

fig.update_layout(width=600, height=400, margin=dict(l=150, r=150, t=20, b=20))
fig = fig_counter.add_title(fig, '一张雷达图', y_pos=0)

if PRODUCTION_PHASE:
    img_file = os.path.join(img_dir, 'img3.svg')
    fig.write_image(img_file, scale=1)
    display_image(img_file)
else:
    fig.show()


#### 利用svgutils工具包进行图片组合

有时一般的数据图表无法满足直观和美观等要求，因此需要通过美术设计来制作比较复杂的图表。这里我们可以使用*svgutils*工具包对多个图片进行组合，以及添加图形、文字等元素，进一步丰富图片的内容。

可参考下面这个例子制作图片。

In [14]:
# general report
general_score = 356
general_percentage = 59
grade_stage = [0, 10, 25, 75, 90]
general_grade = np.sum(np.array(grade_stage)<general_percentage)  

# plot pie
rot = 0
color_maps = ['rgb(255,187,26)', 'rgb(181,229,255)']
label = ['超过','未超过']
pie_value = [general_percentage, 100-general_percentage]
sub_per = str(int(general_percentage))
if general_percentage< 50:
    rot = 360*general_percentage/100
else:
    rot = 0

layout = go.Layout(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)'
)
fig = go.Figure(data=[go.Pie(labels=label, 
                             values=pie_value,
                             hole=.8,
                             marker_colors=color_maps,
                             textinfo='none',
                             rotation = rot
                             )],
                layout=layout)
fig.update_layout(showlegend=False,
                  width=350,
                  height=350,
                 )
img_file = os.path.join(img_dir, 'general_pie_tmp.svg')
fig.write_image(img_file, scale=1)
#display_image(img_file)

heart_loc = [0, 93, 138, 183, 227, 274]
# general plot
compose_fig = sc.Figure(490, 150,
                  sc.SVG(os.path.join(material_dir, 'score_background.svg')),
                  sc.SVG(os.path.join(material_dir, 'xin3.svg')).scale(1.0).move(heart_loc[general_grade], 70),
                  sc.SVG(os.path.join(img_dir, 'general_pie_tmp.svg')).scale(0.8).move(275, -70),
                  sc.Text(str(int(general_percentage))+'%', 392, 120, size=20, font='PingFang SC', weight='bold', color='rgb(255, 187, 26)'),
                  sc.Text(str(int(general_score)), 153, 57, size=25, font='PingFang SC', weight='bold', color='rgb(245, 73, 70)'),
                  #sc.Grid(20, 20),
                  )

compose_fig = fig_counter.add_title(compose_fig, '一张合成的复杂图')

# you can append this part into your code
if PRODUCTION_PHASE:
    compose_fig.save(os.path.join(img_dir, 'general_report_final.svg'))
    display_image(os.path.join(img_dir, 'general_report_final.svg'))
else:
    display(compose_fig)


#### 使用html呈现长表格

在报告中，如果需要呈现比较长的表格，如会跨越多页的表格，可以使用html形式来实现长表格，如下例。

In [15]:
import random
rand_info = []
for i in range(100):
    tmp = []
    for j in range(5):
        tmp.append(random.randint(10, 200))
    rand_info.append(tmp)

tbl_html = sdplot.draw_table(head=['学校名称', '总人数', '男生', '女生', '班级数'],
                             cells=[['A学校', '100人', '60人', '40人', 10],
                                    ['B学校',  '90人', '30人', '60人', 15],
                                    ['C学校',  '80人', '70人', '10人',  6],
                                   ]+rand_info,
                             foot=['合计', '270人', '160人', '110人', 21]
                            )
display(HTML(tbl_html))

学校名称,总人数,男生,女生,班级数
合计,270人,160人,110人,21
A学校,100人,60人,40人,10
B学校,90人,30人,60人,15
C学校,80人,70人,10人,6
123,39,191,180,23
95,55,22,96,193
29,136,52,109,130
60,57,77,24,94
31,144,123,104,129
20,129,38,49,144


## 生成PDF格式的报告文档

在使用Jupyter Notebook完成报告内容后，可以使用**SmartyDoc**提供的处理流程，将ipynb格式的报告文本转换成PDF格式。

### PDF文档转换流程

### 需要准备哪些文件

### 使用printview2插件实现PDF文档生成

### 使用命令行操作实现PDF文档生成